# Carga da Dimensão Tempo

Este notebook realiza a carga da dimensão tempo (dim_tempo).

## Imports

In [ ]:
from spark_config import init_spark
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from datetime import datetime, timedelta

## Start Spark Session

In [ ]:
spark = init_spark("Carga dimensão tempo")

## Variables

In [ ]:
# Definir intervalo de datas
data_inicio = "2000-01-01"
data_fim = "2030-12-31"

# Define caminhos locais onde serão armazenadas as tabelas Delta
base_gold_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/gold"

# Define caminhos das tabelas Delta
delta_path_dim_tempo = f"{base_gold_path}/dim_tempo"

# Feriados fixos brasileiros (formato: mês-dia)
feriados_fixos = [
    "01-01",  # Ano Novo
    "04-21",  # Tiradentes
    "05-01",  # Dia do Trabalho
    "09-07",  # Independência do Brasil
    "10-12",  # Nossa Senhora Aparecida
    "11-02",  # Finados
    "11-15",  # Proclamação da República
    "11-20",  # Consciência Negra
    "12-25",  # Natal
]

# Feriados móveis (adicionar manualmente ou usar biblioteca)
# Para simplificar, vou incluir alguns anos. Em produção, usar biblioteca como workalendar
feriados_moveis = [
    # 2020
    "2020-02-24", "2020-02-25",  # Carnaval
    "2020-04-10",  # Sexta-feira Santa
    "2020-06-11",  # Corpus Christi
    # 2021
    "2021-02-15", "2021-02-16",  # Carnaval
    "2021-04-02",  # Sexta-feira Santa
    "2021-06-03",  # Corpus Christi
    # 2022
    "2022-02-28", "2022-03-01",  # Carnaval
    "2022-04-15",  # Sexta-feira Santa
    "2022-06-16",  # Corpus Christi
    # 2023
    "2023-02-20", "2023-02-21",  # Carnaval
    "2023-04-07",  # Sexta-feira Santa
    "2023-06-08",  # Corpus Christi
    # 2024
    "2024-02-12", "2024-02-13",  # Carnaval
    "2024-03-29",  # Sexta-feira Santa
    "2024-05-30",  # Corpus Christi
    # 2025
    "2025-03-03", "2025-03-04",  # Carnaval
    "2025-04-18",  # Sexta-feira Santa
    "2025-06-19",  # Corpus Christi
    # 2026
    "2026-02-16", "2026-02-17",  # Carnaval
    "2026-04-03",  # Sexta-feira Santa
    "2026-06-04",  # Corpus Christi
    # 2027
    "2027-02-08", "2027-02-09",  # Carnaval
    "2027-03-26",  # Sexta-feira Santa
    "2027-05-27",  # Corpus Christi
    # 2028
    "2028-02-28", "2028-02-29",  # Carnaval
    "2028-04-14",  # Sexta-feira Santa
    "2028-06-15",  # Corpus Christi
    # 2029
    "2029-02-12", "2029-02-13",  # Carnaval
    "2029-03-30",  # Sexta-feira Santa
    "2029-05-31",  # Corpus Christi
    # 2030
    "2030-03-04", "2030-03-05",  # Carnaval
    "2030-04-19",  # Sexta-feira Santa
    "2030-06-20",  # Corpus Christi
]


## Read Source Data

In [ ]:
# Criar lista de datas
start_date = datetime.strptime(data_inicio, "%Y-%m-%d").date()
end_date = datetime.strptime(data_fim, "%Y-%m-%d").date()
date_list = []

current_date = start_date
while current_date <= end_date:
    date_list.append((current_date,))
    current_date += timedelta(days=1)

## Transform Data

In [ ]:
# Criar DataFrame inicial com as datas
df_datas = spark.createDataFrame(date_list, ["dt_dia"])

# Adicionar colunas básicas de data
df_tempo = df_datas.withColumn("nr_dia", F.dayofmonth(F.col("dt_dia"))) \
    .withColumn("nr_dia_ano", F.dayofyear(F.col("dt_dia"))) \
    .withColumn("nr_dia_semana", F.dayofweek(F.col("dt_dia"))) \
    .withColumn("nr_mes", F.month(F.col("dt_dia"))) \
    .withColumn("nr_ano", F.year(F.col("dt_dia")))

# Nome do dia da semana em português
df_tempo = df_tempo.withColumn(
    "nm_dia_semana",
    F.when(F.col("nr_dia_semana") == 1, "Domingo")
    .when(F.col("nr_dia_semana") == 2, "Segunda-feira")
    .when(F.col("nr_dia_semana") == 3, "Terça-feira")
    .when(F.col("nr_dia_semana") == 4, "Quarta-feira")
    .when(F.col("nr_dia_semana") == 5, "Quinta-feira")
    .when(F.col("nr_dia_semana") == 6, "Sexta-feira")
    .when(F.col("nr_dia_semana") == 7, "Sábado")
)

# Nome do mês em português
df_tempo = df_tempo.withColumn(
    "nm_mes",
    F.when(F.col("nr_mes") == 1, "Janeiro")
    .when(F.col("nr_mes") == 2, "Fevereiro")
    .when(F.col("nr_mes") == 3, "Março")
    .when(F.col("nr_mes") == 4, "Abril")
    .when(F.col("nr_mes") == 5, "Maio")
    .when(F.col("nr_mes") == 6, "Junho")
    .when(F.col("nr_mes") == 7, "Julho")
    .when(F.col("nr_mes") == 8, "Agosto")
    .when(F.col("nr_mes") == 9, "Setembro")
    .when(F.col("nr_mes") == 10, "Outubro")
    .when(F.col("nr_mes") == 11, "Novembro")
    .when(F.col("nr_mes") == 12, "Dezembro")
)

# Descrição mês e ano
df_tempo = df_tempo.withColumn(
    "ds_mes_ano",
    F.concat_ws(" ", F.col("nm_mes"), F.col("nr_ano").cast("string"))
)

# Período da semana (segunda a domingo)
df_tempo = df_tempo.withColumn(
    "segunda_feira_semana",
    F.when(F.col("nr_dia_semana") == 1, F.date_sub(F.col("dt_dia"), 6))
    .when(F.col("nr_dia_semana") == 2, F.col("dt_dia"))
    .when(F.col("nr_dia_semana") == 3, F.date_sub(F.col("dt_dia"), 1))
    .when(F.col("nr_dia_semana") == 4, F.date_sub(F.col("dt_dia"), 2))
    .when(F.col("nr_dia_semana") == 5, F.date_sub(F.col("dt_dia"), 3))
    .when(F.col("nr_dia_semana") == 6, F.date_sub(F.col("dt_dia"), 4))
    .when(F.col("nr_dia_semana") == 7, F.date_sub(F.col("dt_dia"), 5))
)

df_tempo = df_tempo.withColumn(
    "domingo_semana",
    F.when(F.col("nr_dia_semana") == 1, F.col("dt_dia"))
    .when(F.col("nr_dia_semana") == 2, F.date_sub(F.col("dt_dia"), -6))
    .when(F.col("nr_dia_semana") == 3, F.date_sub(F.col("dt_dia"), -5))
    .when(F.col("nr_dia_semana") == 4, F.date_sub(F.col("dt_dia"), -4))
    .when(F.col("nr_dia_semana") == 5, F.date_sub(F.col("dt_dia"), -3))
    .when(F.col("nr_dia_semana") == 6, F.date_sub(F.col("dt_dia"), -2))
    .when(F.col("nr_dia_semana") == 7, F.date_sub(F.col("dt_dia"), -1))
)

df_tempo = df_tempo.withColumn(
    "ds_periodo_semana",
    F.concat_ws(
        " a ",
        F.date_format(F.col("segunda_feira_semana"), "dd-MM-yyyy"),
        F.date_format(F.col("domingo_semana"), "dd-MM-yyyy")
    )
).drop("segunda_feira_semana", "domingo_semana")

# Bimestre
df_tempo = df_tempo.withColumn(
    "nr_bimestre",
    F.when(F.col("nr_mes").isin([1, 2]), 1)
    .when(F.col("nr_mes").isin([3, 4]), 2)
    .when(F.col("nr_mes").isin([5, 6]), 3)
    .when(F.col("nr_mes").isin([7, 8]), 4)
    .when(F.col("nr_mes").isin([9, 10]), 5)
    .when(F.col("nr_mes").isin([11, 12]), 6)
).withColumn(
    "ds_bimestre",
    F.concat_ws("", F.col("nr_bimestre").cast("string"), F.lit("º Bimestre"))
)

# Trimestre
df_tempo = df_tempo.withColumn(
    "nr_trimestre",
    F.when(F.col("nr_mes").isin([1, 2, 3]), 1)
    .when(F.col("nr_mes").isin([4, 5, 6]), 2)
    .when(F.col("nr_mes").isin([7, 8, 9]), 3)
    .when(F.col("nr_mes").isin([10, 11, 12]), 4)
).withColumn(
    "ds_trimestre",
    F.concat_ws("", F.col("nr_trimestre").cast("string"), F.lit("º Trimestre"))
)

# Quadrimestre
df_tempo = df_tempo.withColumn(
    "nr_quadrimestre",
    F.when(F.col("nr_mes").isin([1, 2, 3, 4]), 1)
    .when(F.col("nr_mes").isin([5, 6, 7, 8]), 2)
    .when(F.col("nr_mes").isin([9, 10, 11, 12]), 3)
).withColumn(
    "ds_quadrimestre",
    F.concat_ws("", F.col("nr_quadrimestre").cast("string"), F.lit("º Quadrimestre"))
)

# Semestre
df_tempo = df_tempo.withColumn(
    "nr_semeste",
    F.when(F.col("nr_mes").isin([1, 2, 3, 4, 5, 6]), 1)
    .when(F.col("nr_mes").isin([7, 8, 9, 10, 11, 12]), 2)
).withColumn(
    "ds_semestre",
    F.concat_ws("", F.col("nr_semeste").cast("string"), F.lit("º Semestre"))
)

# Último dia do mês
df_tempo = df_tempo.withColumn("dt_ultimo_dia_mes", F.last_day(F.col("dt_dia")))

# Criar coluna para verificar feriados fixos
feriados_fixos_formatados = [f"-{f}" for f in feriados_fixos]

df_tempo = df_tempo.withColumn(
    "mes_dia",
    F.date_format(F.col("dt_dia"), "-MM-dd")
)

# Flag de feriado fixo
feriado_fixo_condition = F.col("mes_dia").isin(feriados_fixos_formatados)

# Flag de feriado móvel
feriados_moveis_list = [datetime.strptime(d, "%Y-%m-%d").date() for d in feriados_moveis]
df_tempo = df_tempo.withColumn(
    "fl_feriado",
    F.when(feriado_fixo_condition, F.lit(True))
    .when(F.col("dt_dia").isin(feriados_moveis_list), F.lit(True))
    .otherwise(F.lit(False))
).drop("mes_dia")

# Flag de fim de semana
df_tempo = df_tempo.withColumn(
    "fl_fim_semana",
    F.when(F.col("nr_dia_semana").isin([1, 7]), F.lit(True)).otherwise(F.lit(False))
)

# Flag de dia útil (não é feriado e não é fim de semana)
df_tempo = df_tempo.withColumn(
    "fl_dia_util",
    F.when((F.col("fl_feriado") == False) & (F.col("fl_fim_semana") == False), F.lit(True))
    .otherwise(F.lit(False))
)

# Criar coluna ano_mes para agrupar
df_tempo = df_tempo.withColumn("ano_mes", F.date_format(F.col("dt_dia"), "yyyy-MM"))

# Ordenar por data descendente dentro de cada mês e filtrar dias úteis
windowSpec = Window.partitionBy("ano_mes").orderBy(F.col("dt_dia").desc())

df_ultimo_util = df_tempo.filter(F.col("fl_dia_util") == True) \
    .withColumn("rn", F.row_number().over(windowSpec)) \
    .filter(F.col("rn") == 1) \
    .select(
        F.col("ano_mes"),
        F.col("dt_dia").alias("dt_ultimo_dia_util_mes_calc")
    )

# Juntar com o dataframe principal
df_tempo = df_tempo.join(df_ultimo_util, on="ano_mes", how="left")

df_tempo = df_tempo.withColumn(
    "dt_ultimo_dia_util_mes",
    F.col("dt_ultimo_dia_util_mes_calc")
).drop("ano_mes", "dt_ultimo_dia_util_mes_calc")

# Adicionar timestamp de inserção
df_tempo = df_tempo.withColumn("ts_insercao", F.current_timestamp())

In [21]:
df_final = df_tempo.select(
    "dt_dia",
    "nr_dia",
    "nr_dia_ano",
    "nr_dia_semana",
    "nm_dia_semana",
    "ds_periodo_semana",
    "nr_mes",
    "nm_mes",
    "ds_mes_ano",
    "nr_bimestre",
    "ds_bimestre",
    "nr_trimestre",
    "ds_trimestre",
    "nr_quadrimestre",
    "ds_quadrimestre",
    "nr_semeste",
    "ds_semestre",
    "nr_ano",
    "dt_ultimo_dia_util_mes",
    "dt_ultimo_dia_mes",
    "fl_feriado",
    "fl_fim_semana",
    "fl_dia_util",
    "ts_insercao"
)

# Estatísticas básicas
print(f"Total de registros: {df_final.count()}")
print(f"Período: {data_inicio} até {data_fim}")

Total de registros: 11323
Período: 2000-01-01 até 2030-12-31


## Write Data

In [22]:
# Escrever na tabela Delta - FORMA CORRETA
df_final.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "false") \
    .save(delta_path_dim_tempo)

print(f"✅ Dados inseridos com sucesso na tabela Delta {delta_path_dim_tempo}")

✅ Dados inseridos com sucesso na tabela Delta D:/Projetos/Jornada_financas_pessoais/data/delta/gold/dim_tempo


## Stop Spark Session

In [23]:
# Encerra a SparkSession
spark.stop()